In [29]:
import transformers
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

In [3]:
df = pd.read_csv('../../dataset/train/train.csv')

In [7]:
a = next(df.iterrows())[1]
a

id                                                                0
sentence          〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...
subject_entity    {'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...
object_entity     {'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...
label                                                   no_relation
source                                                    wikipedia
Name: 0, dtype: object

In [19]:
def change_enti(sub,obj,marker_mode = 'TEM_prunct'):
    if marker_mode == 'TEM_prunct':
        marked_sub = ['@']+['*']+list(sub['type']) + ['*']+list(sub['word'])+['@']
        marked_obj = ['#']+['^']+list(obj['type']) + ['^']+list(obj['word'])+['#']
    elif marker_mode == 'TEM':
        marked_sub = ['<s:']+list(sub['type']) + ['>']+list(sub['word'])+['</s:']+list(sub['type']) + ['>']
        marked_obj = ['<s:']+list(obj['type']) + ['>']+list(obj['word'])+['</s:']+list(obj['type']) + ['>']
    elif marker_mode == "EM":
        marked_sub = ['<subj>']+list(sub['word'])+['</subj']
        marked_obj = ['<obj>']+list(obj['word'])+['</obj']
    return marked_sub, marked_obj

In [40]:
def add_special_enti(df,marker_mode= 'TEM_prunct'):
    def change_enti(sub,obj,marker_mode = 'TEM_prunct'):
        if marker_mode == 'TEM_prunct':
            marked_sub = ['@']+['*']+list(sub['type']) + ['*']+list(sub['word'])+['@']
            marked_obj = ['#']+['^']+list(obj['type']) + ['^']+list(obj['word'])+['#']
        elif marker_mode == 'TEM':
            marked_sub = ['<s:']+list(sub['type']) + ['>']+list(sub['word'])+['</s:']+list(sub['type']) + ['>']
            marked_obj = ['<s:']+list(obj['type']) + ['>']+list(obj['word'])+['</s:']+list(obj['type']) + ['>']
        elif marker_mode == "EM":
            marked_sub = ['<subj>']+list(sub['word'])+['</subj>']
            marked_obj = ['<obj>']+list(obj['word'])+['</obj>']
        return marked_sub, marked_obj
    marked = []
    sub = eval(df['subject_entity'])
    s_s, s_e = sub['start_idx'], sub['end_idx']+1
    obj = eval(df['object_entity'])
    o_s, o_e = obj['start_idx'], obj['end_idx']+1
    marked_sub,marked_obj = change_enti(sub,obj)
    if s_s < o_s:
        marked += df['sentence'][:s_s]
        marked += marked_sub
        marked += df['sentence'][s_e:o_s]
        marked += marked_obj
        marked += df['sentence'][o_e:]
        marked = ''.join(marked)
    else:
        marked += df['sentence'][:o_s]
        marked += marked_obj
        marked += df['sentence'][o_e:s_s]
        marked += marked_sub
        marked += df['sentence'][s_e:]
        marked = ''.join(marked)
    return marked


In [41]:
def tokenizing(dataframe):
    data = []
    for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
        # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
        text = add_special_enti(item)
        # text = '[SEP]'.join([concat_entity, item['sentence']])
        outputs = tokenizer(text, add_special_tokens=True,
                                      padding=False
                                )['input_ids']
        data.append(outputs)
    return data
mx= tokenizing(df)

tokenizing: 100%|██████████| 32470/32470 [00:16<00:00, 1940.09it/s]


In [44]:
tokenizer.decode(mx[3])

'[CLS] 균일가 생활용품점 ( 주 ) @ * ORG * 아성다이소 @ ( 대표 # ^ PER ^ 박정부 # ) 는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다. [SEP]'

In [73]:
ss = pd.Series(mx)
# pd.DataFrame(ss,ss
tmp = ss.apply(lambda x : len(x))
sorted(tmp,reverse=True)

In [64]:
print(df.iloc[13877][['subject_entity','object_entity','label']])
print(df.iloc[13877]['sentence'])
# 문장 긴거 좀 잘라주세요~~ ㅎㅎ

subject_entity    {'word': '서울대학교', 'start_idx': 283, 'end_idx':...
object_entity     {'word': '1946년', 'start_idx': 89, 'end_idx': ...
label                                                   no_relation
Name: 13877, dtype: object
1942년 경성치과전문학교 입학을 하였고 1943년 2월에서 1945년 2월까지 2년간 휴학 끝에 1945년 2월 경성치과전문학교 예과 2년에 복학 후 이듬해 1946년 경성치전(경성치과전문학교)이 서울대학교 치의과대학으로 개편되면서 서울대학교 치의과대학 본과에 편입을 하였는데 바로 그 해, 서울대학교 치과대학 본과 1년 재학 시절인 1946년 교내 연극반 반원으로 연극에 입문하였고 이후 한편으로 서울대학교 치과대학 본과 3년 시절이던 1948년 11월에 육군 군의무관 중위 임관하였고 1950년 2월 서울대학교 치의과대학 학사 학위하고 한 달 지난 1950년 3월에 육군 군의무관 대위 진급하였으며 이후 1950년 6월 25일에서 1951년 11월에 전역을 할 때까지 한국 전쟁에 육군 군의무관으로 참전했으며, 1951년 11월에 육군 군의무관 대위 예편하였다.


In [51]:
class RE_Dataset(torch.utils.data.Dataset):
    """ Dataset 구성을 위한 class."""
    def __init__(self, dataset,labels,tokenizer):
        self.labels = labels
        self.tokenizer = tokenizer
        self.new_tokens = []
        if self.args.input_format == 'entity_marker':
            self.new_tokens = ['<subj>', '</subj>', '<obj>', '</obj>']
        self.tokenizer.add_tokens(self.new_tokens)
        self.dataset = self.tokenizing(dataset)
    def __getitem__(self, idx):
        if len(self.labels) ==0:
            return {'input_ids': torch.LongTensor(self.dataset[idx]['input_ids']).squeeze(0),
                    'attention_mask': torch.LongTensor(self.dataset[idx]['attention_mask']).squeeze(0),
                    'token_type_ids': torch.LongTensor(self.dataset[idx]['token_type_ids']).squeeze(0)
                           }
        else:
            return {'input_ids': torch.LongTensor(self.dataset[idx]['input_ids']).squeeze(0),
                    'attention_mask': torch.LongTensor(self.dataset[idx]['attention_mask']).squeeze(0),
                    'token_type_ids': torch.LongTensor(self.dataset[idx]['token_type_ids']).squeeze(0),
                    'labels' : torch.LongTensor([self.labels[idx]]).squeeze()}
    def __len__(self):
        return len(self.dataset)
    
    def tokenizing(self,dataframe):
        data = []
        for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
            # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
            text = self.add_special_enti(item)
            # text = '[SEP]'.join([concat_entity, item['sentence']])
            outputs = self.tokenizer(text, add_special_tokens=True,
                                          truncation=True,
                                          return_tensors="pt",
                                          padding='max_length',
                                          max_length=256
                                    )
            data.append(outputs)
        return data
    
    def add_special_enti(self,df,marker_mode= 'TEM_prunct'):
        def change_enti(sub,obj,marker_mode = 'TEM_prunct'):
            if marker_mode == 'TEM_prunct':
                marked_sub = ['@']+['*']+list(sub['type']) + ['*']+list(sub['word'])+['@']
                marked_obj = ['#']+['^']+list(obj['type']) + ['^']+list(obj['word'])+['#']
            elif marker_mode == 'TEM':
                marked_sub = ['<s:']+list(sub['type']) + ['>']+list(sub['word'])+['</s:']+list(sub['type']) + ['>']
                marked_obj = ['<s:']+list(obj['type']) + ['>']+list(obj['word'])+['</s:']+list(obj['type']) + ['>']
            elif marker_mode == "EM":
                marked_sub = ['<subj>']+list(sub['word'])+['</subj>']
                marked_obj = ['<obj>']+list(obj['word'])+['</obj>']
            return marked_sub, marked_obj
        marked = []
        sub = eval(df['subject_entity'])
        s_s, s_e = sub['start_idx'], sub['end_idx']+1
        obj = eval(df['object_entity'])
        o_s, o_e = obj['start_idx'], obj['end_idx']+1
        marked_sub,marked_obj = change_enti(sub,obj)
        if s_s < o_s:
            marked += df['sentence'][:s_s]
            marked += marked_sub
            marked += df['sentence'][s_e:o_s]
            marked += marked_obj
            marked += df['sentence'][o_e:]
            marked = ''.join(marked)
        else:
            marked += df['sentence'][:o_s]
            marked += marked_obj
            marked += df['sentence'][o_e:s_s]
            marked += marked_sub
            marked += df['sentence'][s_e:]
            marked = ''.join(marked)
        return marked


In [52]:
import pickle
def label_to_num(label):
    num_label = []
    with open('../dict_label_to_num.pkl', 'rb') as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label
label = label_to_num(df['label'].values)

In [53]:
from tqdm import tqdm
dataset = RE_Dataset(df,label,tokenizer)

tokenizing: 100%|██████████| 32470/32470 [00:21<00:00, 1493.42it/s]


In [28]:
special_token_dict = {'additional_special_tokens' : ['<sub_PER>','</sub_PER>','<sub_ORG>','</sub_ORG>','<sub_LOC>','</sub_LOC>',₩
                                                     '<sub_DAT>','</sub_DAT>','<sub_POH>','</sub_POH>','<sub_NOH>','</sub_NOH>'₩
                                                     '<obj_PER>','</obj_PER>','<obj_ORG>','</obj_ORG>','<obj_LOC>','</obj_LOC>',₩
                                                     '<obj_DAT>','</obj_DAT>','<obj_POH>','</obj_POH>','<obj_NOH>','</obj_NOH>']}
num_added_tokens = tokenizer.add_special_tokens(special_token_dict)

In [36]:
dataset[0]

{'input_ids': tensor([    0,   168, 30985, 14451,  7088,  4586,   169,   793,    32, 21639,
            34,  8373, 14113,  2234,    32,    19, 21639,    34,  1504,  1363,
          2088,    32,    51,  2107,  2341,    34, 29830,    32,    19,    51,
          2107,  2341,    34,   543, 14879,  2440,  6711,   170, 21406, 26713,
          2076, 25145,  5749,   171,  1421,   818,  2073,  4388,  2062,    18,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   